In [1]:
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf

In [2]:
import zipfile
# Upload the zipfile that we want to read
zf = zipfile.ZipFile('C:/Users/mahan/OneDrive/Desktop/Repos/Crime-Predictions/CMPD_Incidents.zip')

# Have Pandas read the csv file in the zip file 
CMPD_df = pd.read_csv(zf.open('CMPD_Incidents.csv'))

CMPD_df.head(5)

C:\Users\mahan\AppData\Local\Temp\ipykernel_19992\3512053027.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  CMPD_df = pd.read_csv(zf.open('CMPD_Incidents.csv'))


,X,Y,YEAR,INCIDENT_REPORT_ID,LOCATION,CITY,STATE,ZIP,X_COORD_PUBLIC,Y_COORD_PUBLIC,...,LOCATION_TYPE_DESCRIPTION,PLACE_TYPE_DESCRIPTION,PLACE_DETAIL_DESCRIPTION,CLEARANCE_STATUS,CLEARANCE_DETAIL_STATUS,CLEARANCE_DATE,HIGHEST_NIBRS_CODE,HIGHEST_NIBRS_DESCRIPTION,OBJECTID,GlobalID
0,1.448107e+06,543688.000087,2021,20210807-1032-01,500 W 4TH ST,CHARLOTTE,NC,28202.0,1448107,543688,...,Outdoors,Public/Non-Residential,Air/Bus/Train Terminal,Open,Open,NaN,23H,All Other Thefts,1,{EAFA7AFE-E804-4C62-A54A-27FE05DC193C}
1,1.406833e+06,499666.999875,2021,20210714-0714-02,14200 PERUGIA WY,CHARLOTTE,NC,28273.0,1406833,499667,...,Parking Lot,Residential,Apartment/Duplex Private Res,Open,Open,NaN,23F,Theft From Motor Vehicle,2,{3A654D83-2451-4CD1-8DDF-D4DC033AFDAA}
2,1.439801e+06,548658.999988,2017,20170521-1457-02,3000 RUSH AV,CHARLOTTE,NC,28208.0,1439801,548659,...,Outdoors,Residential,Private Residence,Open,Open,NaN,290,Damage/Vandalism Of Property,3,{041746DD-6C3E-41AD-B4CB-7D98FBF99F95}
3,1.449394e+06,572029.000163,2022,20221028-1713-02,4900 SUNSET RD,CHARLOTTE,NC,28269.0,1449394,572029,...,Indoors,Commercial Place,Hotel/Motel,Open,Open,NaN,11D,Forcible Fondling,4,{C122DBB6-928D-49AA-BB56-6D73D609E03D}
4,1.432559e+06,550084.000090,2020,20201214-0857-00,4600 TUCKASEEGEE RD,CHARLOTTE,NC,NaN,1432559,550084,...,Indoors,Commercial Place,Day Care,Open,Open,NaN,220,Burglary/B&E,5,{0BA4BB41-45FF-425F-A678-7A4676379EA9}


In [3]:
# Find the names of the columns so we can remove some of the unimportant ones
CMPD_df.columns

Index(['X', 'Y', 'YEAR', 'INCIDENT_REPORT_ID', 'LOCATION', 'CITY', 'STATE',
       'ZIP', 'X_COORD_PUBLIC', 'Y_COORD_PUBLIC', 'LATITUDE_PUBLIC',
       'LONGITUDE_PUBLIC', 'DIVISION_ID', 'CMPD_PATROL_DIVISION', 'NPA',
       'DATE_REPORTED', 'DATE_INCIDENT_BEGAN', 'DATE_INCIDENT_END',
       'ADDRESS_DESCRIPTION', 'LOCATION_TYPE_DESCRIPTION',
       'PLACE_TYPE_DESCRIPTION', 'PLACE_DETAIL_DESCRIPTION',
       'CLEARANCE_STATUS', 'CLEARANCE_DETAIL_STATUS', 'CLEARANCE_DATE',
       'HIGHEST_NIBRS_CODE', 'HIGHEST_NIBRS_DESCRIPTION', 'OBJECTID',
       'GlobalID'],
      dtype='object')

In [4]:
#Check the data types
CMPD_df.dtypes

X                            float64
Y                            float64
YEAR                           int64
INCIDENT_REPORT_ID            object
LOCATION                      object
CITY                          object
STATE                         object
ZIP                           object
X_COORD_PUBLIC                 int64
Y_COORD_PUBLIC                 int64
LATITUDE_PUBLIC              float64
LONGITUDE_PUBLIC             float64
DIVISION_ID                   object
CMPD_PATROL_DIVISION          object
NPA                            int64
DATE_REPORTED                 object
DATE_INCIDENT_BEGAN           object
DATE_INCIDENT_END             object
ADDRESS_DESCRIPTION           object
LOCATION_TYPE_DESCRIPTION     object
PLACE_TYPE_DESCRIPTION        object
PLACE_DETAIL_DESCRIPTION      object
CLEARANCE_STATUS              object
CLEARANCE_DETAIL_STATUS       object
CLEARANCE_DATE                object
HIGHEST_NIBRS_CODE            object
HIGHEST_NIBRS_DESCRIPTION     object
O

In [5]:
# Create a new dataframe with the data you want to train
# There were many good variables, but many were repetitive or superfluous for the amount of data that needs to be trained
cmpd_df = CMPD_df[['YEAR', 'ZIP', 'DIVISION_ID', 'NPA', 'DATE_REPORTED', 'PLACE_DETAIL_DESCRIPTION', 'HIGHEST_NIBRS_CODE','CLEARANCE_STATUS']]
print(cmpd_df.head())

   YEAR      ZIP DIVISION_ID  NPA           DATE_REPORTED  \
0  2021  28202.0          01  476  2021/08/07 00:00:00+00   
1  2021  28273.0          21   82  2021/07/14 00:00:00+00   
2  2017  28208.0          02  293  2017/05/21 00:00:00+00   
3  2022  28269.0          11  125  2022/10/28 00:00:00+00   
4  2020      NaN          27  199  2020/12/14 00:00:00+00   

       PLACE_DETAIL_DESCRIPTION HIGHEST_NIBRS_CODE CLEARANCE_STATUS  
0        Air/Bus/Train Terminal                23H             Open  
1  Apartment/Duplex Private Res                23F             Open  
2             Private Residence                290             Open  
3                   Hotel/Motel                11D             Open  
4                      Day Care                220             Open  


In [6]:
# Find the minimum and maximum date so we know how much time is covered in the dataset
min_date = cmpd_df['DATE_REPORTED'].min()
max_date = cmpd_df['DATE_REPORTED'].max()
print(f'Earliest date in dataset: {min_date}\nLatest date in dataset: {max_date}')

Earliest date in dataset: 2017/01/01 00:00:00+00
Latest date in dataset: 2024/05/20 00:00:00+00


In [7]:
#Check the data types
cmpd_df.dtypes

YEAR                         int64
ZIP                         object
DIVISION_ID                 object
NPA                          int64
DATE_REPORTED               object
PLACE_DETAIL_DESCRIPTION    object
HIGHEST_NIBRS_CODE          object
CLEARANCE_STATUS            object
dtype: object

In [8]:
#Change the values in the 'CLEARANCE STATUS' to 0 or 1
#Define the mapping 
mapping = {'Open': True, 'Cleared by Arrest': False, 'Cleared by Arrest by Another Agency': False, 'Exceptionally Cleared': False, 'Unfounded': False}

# Apply mapping to the 'CLEARANCE_STATUS' column and fill NaN values with a default value (e.g., False)
CMPD_df['CLEARANCE_STATUS'] = CMPD_df['CLEARANCE_STATUS'].map(mapping).fillna(False)

#check to make sure the above worked
CMPD_df.head(5)

#Convert the 'CLEARANCE_STATUS' column into an array 

status_array = CMPD_df['CLEARANCE_STATUS'].values
print(status_array)


[ True  True  True ...  True False False]


In [9]:
# Change DATE_REPORTED date time to only the year and month so there are less columns to train. 
# This could decrease the amount of columns from potentially 2,500+ to 84 for just dates. 
# This can be optimized if we want to add more specific dates later.
cmpd_df['DATE_REPORTED'] = pd.to_datetime(cmpd_df['DATE_REPORTED'])

# Change the datetime format to display only the year and month
cmpd_df['DATE_REPORTED_Y/M'] = cmpd_df['DATE_REPORTED'].dt.strftime('%Y-%m')

# Display the DataFrame
print(cmpd_df.head())
print(cmpd_df.shape)

C:\Users\mahan\AppData\Local\Temp\ipykernel_19992\3478752487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmpd_df['DATE_REPORTED'] = pd.to_datetime(cmpd_df['DATE_REPORTED'])


   YEAR      ZIP DIVISION_ID  NPA             DATE_REPORTED  \
0  2021  28202.0          01  476 2021-08-07 00:00:00+00:00   
1  2021  28273.0          21   82 2021-07-14 00:00:00+00:00   
2  2017  28208.0          02  293 2017-05-21 00:00:00+00:00   
3  2022  28269.0          11  125 2022-10-28 00:00:00+00:00   
4  2020      NaN          27  199 2020-12-14 00:00:00+00:00   

       PLACE_DETAIL_DESCRIPTION HIGHEST_NIBRS_CODE CLEARANCE_STATUS  \
0        Air/Bus/Train Terminal                23H             Open   
1  Apartment/Duplex Private Res                23F             Open   
2             Private Residence                290             Open   
3                   Hotel/Motel                11D             Open   
4                      Day Care                220             Open   

  DATE_REPORTED_Y/M  
0           2021-08  
1           2021-07  
2           2017-05  
3           2022-10  
4           2020-12  
(659886, 9)


C:\Users\mahan\AppData\Local\Temp\ipykernel_19992\3478752487.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cmpd_df['DATE_REPORTED_Y/M'] = cmpd_df['DATE_REPORTED'].dt.strftime('%Y-%m')


In [10]:
# Drop the original DATE_REPORTED column and null values

# Declare new df with dropped nulls 
cleanedish_cmpd_df = cmpd_df.dropna()

# Declare final df after dropping DATE_REPORTED column 
cleaned_cmpd_df = cleanedish_cmpd_df.drop(columns=['DATE_REPORTED'])

# Verify column was dropped
print(cleaned_cmpd_df.head())
# Verify that null rows were dropped
print(cleaned_cmpd_df.shape)

   YEAR      ZIP DIVISION_ID  NPA      PLACE_DETAIL_DESCRIPTION  \
0  2021  28202.0          01  476        Air/Bus/Train Terminal   
1  2021  28273.0          21   82  Apartment/Duplex Private Res   
2  2017  28208.0          02  293             Private Residence   
3  2022  28269.0          11  125                   Hotel/Motel   
6  2017  28215.0          07  271             Private Residence   

  HIGHEST_NIBRS_CODE CLEARANCE_STATUS DATE_REPORTED_Y/M  
0                23H             Open           2021-08  
1                23F             Open           2021-07  
2                290             Open           2017-05  
3                11D             Open           2022-10  
6                220             Open           2017-08  
(448619, 8)


In [19]:
# Encode categorical variables
#needs_encoding = ['PLACE_DETAIL_DESCRIPTION','HIGHEST_NIBRS_CODE','DATE_REPORTED_Y/M']
final_cmpd_df = pd.get_dummies(cleaned_cmpd_df, columns= ['CLEARANCE_STATUS'])
final_cmpd_df.head()

,YEAR,ZIP,DIVISION_ID,NPA,PLACE_DETAIL_DESCRIPTION,HIGHEST_NIBRS_CODE,DATE_REPORTED_Y/M,CLEARANCE_STATUS_Cleared by Arrest,CLEARANCE_STATUS_Cleared by Arrest by Another Agency,CLEARANCE_STATUS_Exceptionally Cleared,CLEARANCE_STATUS_Open,CLEARANCE_STATUS_Unfounded
0,2021,28202.0,01,476,Air/Bus/Train Terminal,23H,2021-08,False,False,False,True,False
1,2021,28273.0,21,82,Apartment/Duplex Private Res,23F,2021-07,False,False,False,True,False
2,2017,28208.0,02,293,Private Residence,290,2017-05,False,False,False,True,False
3,2022,28269.0,11,125,Hotel/Motel,11D,2022-10,False,False,False,True,False
6,2017,28215.0,07,271,Private Residence,220,2017-08,False,False,False,True,False


In [20]:
final_cmpd_df.dtypes

YEAR                                                     int64
ZIP                                                     object
DIVISION_ID                                             object
NPA                                                      int64
PLACE_DETAIL_DESCRIPTION                                object
HIGHEST_NIBRS_CODE                                      object
DATE_REPORTED_Y/M                                       object
CLEARANCE_STATUS_Cleared by Arrest                        bool
CLEARANCE_STATUS_Cleared by Arrest by Another Agency      bool
CLEARANCE_STATUS_Exceptionally Cleared                    bool
CLEARANCE_STATUS_Open                                     bool
CLEARANCE_STATUS_Unfounded                                bool
dtype: object

In [28]:
# Split data into features and target variable


#X = final_cmpd_df.copy()
#X.drop(['CLEARANCE_STATUS'], axis=1)

y = final_cmpd_df['CLEARANCE_STATUS_Open'].astype(int)


In [34]:
#final_cmpd_df.head()
X

,YEAR,ZIP,DIVISION_ID,NPA,PLACE_DETAIL_DESCRIPTION,HIGHEST_NIBRS_CODE,DATE_REPORTED_Y/M,CLEARANCE_STATUS_Cleared by Arrest,CLEARANCE_STATUS_Cleared by Arrest by Another Agency,CLEARANCE_STATUS_Exceptionally Cleared,CLEARANCE_STATUS_Open,CLEARANCE_STATUS_Unfounded
0,2021,28202.0,01,476,Air/Bus/Train Terminal,23H,2021-08,False,False,False,True,False
1,2021,28273.0,21,82,Apartment/Duplex Private Res,23F,2021-07,False,False,False,True,False
2,2017,28208.0,02,293,Private Residence,290,2017-05,False,False,False,True,False
3,2022,28269.0,11,125,Hotel/Motel,11D,2022-10,False,False,False,True,False
6,2017,28215.0,07,271,Private Residence,220,2017-08,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
659880,2020,28202.0,01,476,Apartment/Duplex Private Res,13B,2020-11,True,False,False,False,False
659882,2023,28211.0,16,370,Apartment/Duplex Private Res,13B,2023-01,True,False,False,False,False
659883,2021,28273.0,21,92,Hotel/Motel,23F,2021-11,False,False,False,True,False
659884,2019,28270.0,17,287,Grocery Store/Supermarket,23H,2019-07,True,False,False,False,False


In [30]:
# Set the data rate limit directly
#import os
#os.environ['NotebookApp.iopub_data_rate_limit'] = '1000000000'

#def find_string_values(final_cmpd_df):
    #string_values = []
    #for col in final_cmpd_df.columns:
       # for i, val in enumerate(final_cmpd_df[col]):
            #if isinstance(val, str):
                #string_values.append((col, i, val))
    #return string_values

# Call the function on your DataFrame
#result = find_string_values(final_cmpd_df)
#print(result)

In [31]:
# Import sklearn to spilt the dataset into training and testing data
from sklearn.model_selection import train_test_split

# Assuming X and y are your features and target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)


In [32]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ValueError: could not convert string to float: '282-5'

In [ ]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()